In [1]:
import numpy as np
import pandas as pd
from glob import glob
from random import shuffle
from tqdm import tqdm
import swifter

C:\Users\Matheus\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Matheus\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Matheus\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv('simulated_games/simulated_games_0/game_999.csv')

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,2,0
1,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,4,4
2,2,0,0,2,0,0,0,4,0,0,0,2,0,0,0,0,4
3,4,0,0,0,4,0,0,2,2,0,0,0,0,0,0,0,8
4,0,0,0,0,0,0,0,0,8,0,0,0,2,2,0,2,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,0,4,16,2,2,32,2048,16,2,128,64,256,2,8,2,16,23188
1360,4,4,16,2,2,32,2048,16,0,128,64,256,4,8,2,16,23192
1361,4,8,16,2,2,32,2048,16,0,128,64,256,4,8,2,16,23200
1362,4,8,16,2,2,32,2048,16,0,128,64,256,4,8,2,16,23200


In [4]:
def get_board_state(df_row):
    row1 = df_row[0:4]
    row2 = df_row[4:8]
    row3 = df_row[8:12]
    row4 = df_row[12:16]
    board = np.array([row1, row2, row3, row4], dtype=object)
    return board

In [5]:
boards = df.apply(get_board_state, axis=1)
labels = df['16']

In [6]:
filepaths = glob('simulated_games/simulated_games_0/*.csv')

In [10]:
# Generator based dataloader
def dataloader(filepaths, batch_size=8):
    X, y = [], []
    
    while True:
        
        # Shuffle the file order before each iteration
        random.shuffle(filepaths)
        
        for filepath in filepaths:
            pass

In [8]:
df = pd.DataFrame()
for filepath in tqdm(filepaths):
    tmp = pd.read_csv(filepath)
    df = pd.concat([df, tmp], ignore_index=True)
df.shape

100%|██████████████████████████████████████████████████████████████████████████████| 1198/1198 [01:38<00:00, 12.13it/s]


In [9]:
%%time
boards = df.swifter.progress_bar(False).apply(get_board_state, axis=1)
labels = df['16']

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Wall time: 3min 40s


In [62]:
# Check the max score over all positions over all games to know the max depth needed for the NN
max_depth = int(np.log2(df.drop('16', axis=1).max().max())) + 1

In [68]:
def encode_board(board):
    encoding = np.zeros((max_depth,4,4), dtype=int)
    
    for row, row_values in enumerate(board):
        for col, value in enumerate(row_values):
            if value == 0:
                depth = 0
            else:
                depth = int(np.log2(value))
            
            encoding[depth][row][col] = 1
            
    return encoding

In [78]:
%%time
encodings = boards.swifter.progress_bar(False).apply(encode_board, axis=1)

Wall time: 1min 14s


In [79]:
encodings

0          [[[0, 0, 1, 1], [1, 1, 1, 1], [1, 1, 0, 1], [1...
1          [[[0, 1, 1, 0], [1, 1, 1, 1], [0, 1, 1, 1], [1...
2          [[[0, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 1], [0...
3          [[[0, 1, 1, 0], [0, 0, 1, 1], [1, 1, 1, 1], [1...
4          [[[1, 1, 0, 0], [1, 1, 0, 0], [1, 0, 1, 1], [1...
                                 ...                        
1652409    [[[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0...
1652410    [[[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0...
1652411    [[[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0...
1652412    [[[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0...
1652413    [[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0...
Length: 1652414, dtype: object

In [80]:
boards

0          [[2, 2, 0, 0], [0, 0, 0, 0], [0, 0, 4, 0], [0,...
1          [[4, 0, 0, 2], [0, 0, 0, 0], [4, 0, 0, 0], [0,...
2          [[8, 0, 0, 2], [0, 0, 0, 0], [0, 0, 0, 0], [2,...
3          [[8, 0, 0, 2], [2, 4, 0, 0], [0, 0, 0, 0], [0,...
4          [[0, 0, 8, 2], [0, 0, 2, 4], [0, 2, 0, 0], [0,...
                                 ...                        
1652409    [[0, 4, 16, 2], [2, 32, 2048, 16], [2, 128, 64...
1652410    [[4, 4, 16, 2], [2, 32, 2048, 16], [0, 128, 64...
1652411    [[4, 8, 16, 2], [2, 32, 2048, 16], [0, 128, 64...
1652412    [[4, 8, 16, 2], [2, 32, 2048, 16], [0, 128, 64...
1652413    [[2, 8, 16, 2], [4, 32, 2048, 16], [2, 128, 64...
Length: 1652414, dtype: object